In [1]:
# Importing required libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import warnings
warnings.filterwarnings('ignore')

### Model Creation

In [2]:
# Initialize the Sequential model
model = Sequential()

# First layer: Convolutional Layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))

# Second layer: Max Pooling Layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third layer: Flatten Layer
model.add(Flatten())

# Fourth layer: Dense Layer
model.add(Dense(64, activation='relu'))

# Final layer: Output Layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=SGD(learning_rate=0.002, momentum=0.8), loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11215873 (42.79 MB)
Trainable params: 11215873 (42.79 MB)
Non-trainable params: 0 (0.00 Byte)
______________

#### Question 1: Since we have a binary classification problem, what is the best loss function for us?

- `mean squared error`
- `binary crossentropy`
- `categorical crossentropy`
- `cosine similarity`

**Answer:** Binary Crossentropy

#### Question 2: What's the number of parameters in the convolutional layer of our model? You can use the `summary` method for that:
- 1
- 65
- 896
- 11214912

**Answer:** 896

### Generators and Training

In [3]:
train_path = 'homework_data/train'
test_path = 'homework_data/test'

# Define the ImageDataGenerator for rescaling
datagen = ImageDataGenerator(rescale=1./255)

# Setup the train and test generators
train_generator = datagen.flow_from_directory(
    train_path,              
    target_size=(150, 150),  # Assuming the target size is (150, 150)
    batch_size=20,
    class_mode='binary',     # For binary classification
    shuffle=True             # Shuffling the data
)

test_generator = datagen.flow_from_directory(
    test_path,               
    target_size=(150, 150),  # Assuming the target size is (150, 150)
    batch_size=20,
    class_mode='binary',     # For binary classification
    shuffle=True             # Shuffling the data
)

# Fit the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Found 3678 images belonging to 2 classes.
Found 918 images belonging to 2 classes.
Epoch 1/10
184/184 [==============================] - 50s 270ms/step - loss: 0.6923 - accuracy: 0.5372 - val_loss: 0.6836 - val_accuracy: 0.5370
Epoch 2/10
184/184 [==============================] - 43s 236ms/step - loss: 0.6757 - accuracy: 0.5609 - val_loss: 0.6420 - val_accuracy: 0.6166
Epoch 3/10
184/184 [==============================] - 59s 319ms/step - loss: 0.6593 - accuracy: 0.5957 - val_loss: 0.6188 - val_accuracy: 0.6612
Epoch 4/10
184/184 [==============================] - 50s 271ms/step - loss: 0.6322 - accuracy: 0.6349 - val_loss: 0.5937 - val_accuracy: 0.6688
Epoch 5/10
184/184 [==============================] - 52s 282ms/step - loss: 0.5971 - accuracy: 0.6805 - val_loss: 0.5646 - val_accuracy: 0.7244
Epoch 6/10
184/184 [==============================] - 49s 268ms/step - loss: 0.5540 - accuracy: 0.7276 - val_loss: 0.5364 - val_accuracy: 0.7429
Epoch 7/10
184/184 [===========================

In [4]:
# Extract the training accuracy and loss from the history object
training_accuracy = history.history['accuracy']
training_loss = history.history['loss']

# Calculate the median of training accuracy
median_accuracy = np.median(training_accuracy)

# Calculate the standard deviation of training loss
std_loss = np.std(training_loss)

# Print the results
print(f"Median of Training Accuracy: {median_accuracy}")
print(f"Standard Deviation of Training Loss: {std_loss}")

Median of Training Accuracy: 0.7040511071681976
Standard Deviation of Training Loss: 0.08126418310341978


#### Question 3: What is the median of training accuracy for all the epochs for this model?

- 0.20
- 0.40
- 0.60
- 0.80

**Answer:** 0.80

#### Question 4: What is the standard deviation of training loss for all the epochs for this model?

- 0.031
- 0.061
- 0.091
- 0.131

**Answer:** 0.091

### Data Augmentation

In [5]:
# Define the ImageDataGenerator with data augmentation for the training set
train_datagen_augmented = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Setup the augmented train generator
train_generator_augmented = train_datagen_augmented.flow_from_directory(
    train_path,              
    target_size=(150, 150),  # Assuming the target size is (150, 150)
    batch_size=20,
    class_mode='binary',     # For binary classification
    shuffle=True             # Shuffling the data
)

# Continue training the model for 10 more epochs
history_augmented = model.fit(
    train_generator_augmented,
    epochs=10,
    validation_data=test_generator
)

Found 3678 images belonging to 2 classes.
Epoch 1/10
184/184 [==============================] - 59s 319ms/step - loss: 0.5071 - accuracy: 0.7708 - val_loss: 0.4928 - val_accuracy: 0.7636
Epoch 2/10
184/184 [==============================] - 59s 318ms/step - loss: 0.5037 - accuracy: 0.7626 - val_loss: 0.4929 - val_accuracy: 0.7560
Epoch 3/10
184/184 [==============================] - 61s 329ms/step - loss: 0.4905 - accuracy: 0.7716 - val_loss: 0.4783 - val_accuracy: 0.7745
Epoch 4/10
184/184 [==============================] - 58s 314ms/step - loss: 0.4877 - accuracy: 0.7727 - val_loss: 0.5174 - val_accuracy: 0.7429
Epoch 5/10
184/184 [==============================] - 59s 319ms/step - loss: 0.4865 - accuracy: 0.7792 - val_loss: 0.4729 - val_accuracy: 0.7778
Epoch 6/10
184/184 [==============================] - 55s 297ms/step - loss: 0.4819 - accuracy: 0.7781 - val_loss: 0.5172 - val_accuracy: 0.7538
Epoch 7/10
184/184 [==============================] - 55s 297ms/step - loss: 0.4726 - ac

In [6]:
# Extract the test loss and accuracy from the history_augmented object
test_loss = history_augmented.history['val_loss']
test_accuracy = history_augmented.history['val_accuracy']

# Calculate the mean of test loss for all the epochs
mean_test_loss = np.mean(test_loss)

# Calculate the average test accuracy for the last 5 epochs (from epoch 6 to 10)
average_test_accuracy_last_5 = np.mean(test_accuracy[-5:])

# Print the results
print(f"Mean of Test Loss for All Epochs: {mean_test_loss}")
print(f"Average Test Accuracy for Last 5 Epochs: {average_test_accuracy_last_5}")

Mean of Test Loss for All Epochs: 0.48915664553642274
Average Test Accuracy for Last 5 Epochs: 0.7725490093231201


#### Question 5: What is the mean of test loss for all the epochs for the model trained with augmentations?

- 0.18
- 0.48
- 0.78
- 0.1080.108

**Answer:** 0.48

#### Question 6: What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

- 0.38
- 0.58
- 0.78
- 0.98

**Answer:** 0.78